# CodeLab 0: Twitter API

Questions? Message @daniel.zou on Slack! :>

## HTTP Requests Basics

Let's learn how to use the Twitter API. For simplicity, we will use the Twitter API directly. There are many Python API wrappers like Tweepy that give us convenient functions for using the Twitter API. In this notebook, we will not learn a specific API wrapper.

Our objective to get send a GET request to Twitter's servers. Twitter will interpret our request, do some logic on its end, and send us a response.

For starters, let's actually send a GET request to a familiar server: ```https://google.com/```. We'll use the python `requests` package, which is a very common library used to send HTTP requests in Python. 

In [1]:
import requests
r = requests.get("https://google.com/")
r

<Response [200]>

We received a status code 200, which means success. For your own learning, you can check out all other possible status codes [here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).

We can grab the data from our response using `r.text`. Let's do that now and see what it looks like.

In [2]:
r.text[:300] # Print only the first 300 characters for brevity

'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="en"><head><meta content="Search the world\'s information, including webpages, images, videos and more. Google has many special features to help you find exactly what you\'re looking for." name="description"><meta content="noo'

For this particular response, Google sends us HTML which our browsers can read and render as a webpage. Let's render this HTML into our notebook:

In [5]:
from IPython.core.display import display, HTML
display(HTML(r.text))

,,Advanced searchLanguage tools


## Sending Requests to the Twitter API

The Twitter API can be accessed by a URL, just like `Google.com`. Specifically, we want to send a GET request to `https://api.twitter.com/1.1/ search/tweets.json?q=SFMTA`. We already know how to send GET requests, so let's just do that now.

In [6]:
requests.get("https://api.twitter.com/1.1/search/tweets.json?q=SFMTA&tweet_mode=extended")

<Response [400]>

Unfortunately, we got a 400 response, which means failure. This is because Twitter restricts their API to registered developers. So first, we have to get some developer credentials by following the instructions [here](https://developer.twitter.com/en/apply-for-access.html).

We will place our credentials in a file called `creds.txt`, in the same directory as this notebook, in this format:
<br></br>`[ConsumerAPIkeys.APIkey]`<br></br>
`[ConsumerAPIkeys.APIsecretkey]`<br></br>
`[AccessToken&AccessSecret.AccessToken]`<br></br>
`[AccessToken&AccessSecret.AccessTokenSecret]`<br></br>
Where we replace `[something]` with its value (including deleting the brackets)<br></br>
Make sure you separate each value with a newline

In [86]:
with open('twitter_credentials.json') as cred_data:
    info = json.load(cred_data)
    consumer_key = info['CONSUMER_KEY']
    consumer_secret = info['CONSUMER_SECRET']
    access_key = info['ACCESS_KEY']
    access_secret = info['ACCESS_SECRET']

Twitter uses something called OAuth1, which is an open standard for accessing restricted data. `requests` has some features to help us manage OAuth1. However, these are not installed by default, so let's install it if necessary.

In [54]:
!pip install requests requests_oauthlib

In [94]:
from requests_oauthlib import OAuth1
auth = OAuth1(consumer_key, consumer_secret, access_key, access_secret)

Finally, let's make our GET request like before, but pass into our `Auth` object to the method as well.

In [95]:
r = requests.get("https://api.twitter.com/1.1/search/tweets.json?q=SFMTA&tweet_mode=extended", auth=auth)
r.text[:200] # Let's only print the first 200 characters for brevity

'{"errors":[{"code":89,"message":"Invalid or expired token."}]}'

Our response data actually is in a format called JSON, which we can convert easily into a Python dictionary using the json package.

In [49]:
import json
response_dict = json.loads(r.text)
print(response_dict.keys())

dict_keys(['errors'])


By inspecting the raw response text above, it looks like our tweets are in the "statuses" key, which maps to an array value. Looking at the array, we see that each tweet is an object in the array. Let's just grab the first tweet.

In [23]:
tweet = response_dict["statuses"][0]
tweet

KeyError: 'statuses'

Wow! Twitter gives us a lot of interesting data to work with, some of which might be handy later. For now, let's just extract the message out of this object using the "full_text" field.

In [46]:
tweet["full_text"]

'Next bus is 42 minutes farther up the line. How is this acceptable @sfmta_muni? https://t.co/4bXOX5ctbT'

Okay fantastic! We've successfully made a request to the Twitter API search endpoint directly supplying the "SFMTA" query string. With this foundation, we can start pulling and saving tweets so that we can do analysis on them later. There's also tons of interesting and useful metadata that Twitter supplies us that may help us do a better job.

Thanks for sticking with it and reaching the end of the CodeLab. I encourage you to continue working through the CodeLabs so that you can get up to speed and start contributing to #nltweets as soon as possible! :)